## Практикум Python

### Тема 5. Замыкания, декораторы

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Содержание:**
1. Замыкания
2. Декораторы
3. Обработка исключений

#### Повторение

In [ ]:
# Что выведется в резульате работы данного фрагмента программы?

x = 15

def foo():
    x = 30

foo()
print(x)

Как изменить поведение кода?

In [ ]:
x = 15

def foo():
    x = 30

foo()
print(x)

In [ ]:
# Какой будет результат выполнения программы?
x = 15

def foo():
    print(x)
    
    x = 30
    print(x)
    
foo()

### Замыкания

**Замыкание** (closure) — это функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся ее параметрами.

Существования замыканий следует из правила LEGB, возможности оперировать с функциями как обьектами и того что области видимости в Питоне - статические.

In [2]:
multipliers = []

for m in range(5):
    multipliers.append(lambda x: x * m)

print([multipliers[i](5) for i in range(5)])

[20, 20, 20, 20, 20]


В качестве элемента в список `multipliers` добавляются функции

In [3]:
multipliers

[<function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>]

**Вопрос:** Почему все элементы получились одинаковыми?

Перепишем в виде функции, чтобы стало понятнее:

In [4]:
multipliers = []

def foo(x):
    return x * m

for m in range(5):
    multipliers.append(foo)

print(m)
print([multipliers[i](5) for i in range(5)])

4
[20, 20, 20, 20, 20]


Чтобы мы могли создавать функции с сохранением внешних переменных, используют замыкания:

In [5]:
def foo():
    x = 3
    def bar():
        print(x)
    x = 5
    return bar

bar = foo()
bar()

x = 9
bar()

5
5


Еще один пример (более осмысленный):

In [6]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

add_two = make_adder(2)

print(add_two(5))
print(add_two(7))

7
9


Вызов функции `add_two` с аргументом 5 можно переписать иначе

In [10]:
make_adder(2)(5)

7

В первых скобках аргумент для внешней функции `male_adder`, в вторых скобках аргумент для `adder`

In [11]:
def make_adder(x):
    print('x =', x)
    def adder(y):
        print('y =', y)
        return x + y
    return adder

make_adder(2)(5)

x = 2
y = 5


7

**Задание** Перепишите первый пример так, чтобы внешняя переменная сохранилась

In [ ]:
'''multipliers = []

def foo(x):
    return x * m

for m in range(5):
    multipliers.append(foo)

print(m)
print([multipliers[i](5) for i in range(5)])'''

# your code here


Функции могут замыкать одинаковые переменные

In [7]:
def cell(value = 0):
    def Get():
        return value
    
    def Set(new_value):
        nonlocal value
        value = new_value
        return value
    
    return Get, Set

Get, Set = cell(10)
print(Get())

Set(20)
print(Get())

10


20

20


Посмотрим, что внутри замыкания. `Get.__closure__` хранит все замкнутые переменные. Можем залезть в отдельную переменную с помощью `Get.__closure__[0].cell_contents`:

In [8]:
print(Get.__closure__)
print(Get.__closure__[0].cell_contents)

(<cell at 0x0000027FA08C2460: int object at 0x0000027F9BB96B90>,)
20


Переменная представлена в виде класса `cell` с единственным полем `cell_contents`. Убедимся, что замкнутые переменные для `Get` и `Set` одинаковые:

In [9]:
print(Get.__closure__ == Set.__closure__)
print(Get.__closure__[0] is Set.__closure__[0])

True
True


### Декораторы

Замыкания могут использоваться как способ быстро изменить поведение функции:

In [12]:
import sys

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint([1, 2, 3])

[1, 2, 3]


print is deprecated


Такой синтаксис немного неудобный, поэтому в Python есть более удобный способ реализовать декоратор:

In [13]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper


@deprecated
def show(x):
    print(x)

show([1, 2, 3])

[1, 2, 3]


show is deprecated


Однако такое использование влечет за собой проблему:

In [14]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

wrapper
None


**Вопрос:** В чём заключается проблема?

**Решение 1.** Просто перенести нужные атрибуты

In [15]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


**Решение 2.** Использовать functools

In [16]:
import functools

def deprecated(func):
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


#### Декораторы с аргументами

В Python так же можно создавать декораторы с дополнительными аргументами:

In [17]:
def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('{} called with args {}, kwargs {}!'.format(func.__name__, args, kwargs), file = dest)
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace(sys.stdout) 
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

f called with args ('Hi!',), kwargs {'test': 42}!
f called with args ('Hi!', 21.0), kwargs {}!
f called with args ('Hi!', 10.5), kwargs {}!
f called with args ('Hi!', 5.25), kwargs {}!
f called with args ('Hi!', 2.625), kwargs {}!
f called with args ('Hi!', 1.3125), kwargs {}!
f called with args ('Hi!', 0.65625), kwargs {}!


**Задача** Написать декоратор `once(func)`, который вызывает функцию только один раз.

In [ ]:
def once(func):
    # your code here
    pass



@once
def foo():
    print('Hi!')

foo()
foo()

Декораторам необязательно быть функциями:

In [18]:
from collections import Counter 

class Register(object):
    def __init__(self):
        self.stat = Counter()
        
    def __call__(self, func):
        nm = func.__name__
        def wrapper(*args, **kwrags):
            self.stat[nm] += 1
            return func(*args, **kwrags)
        return wrapper
    
    def __str__(self):
        result = 'fname\tcallcount\n'
        for name, count in self.stat.items():
            result += '{}:\t{}\n'.format(name, count)
        return result
    
register = Register()

In [19]:
@register
def f(x):
    return x 

@register
def q(x):
    return q

f(1), q(2), q(4)
q(2), f(5)
print(register)

(1,
 <function __main__.Register.__call__.<locals>.wrapper(*args, **kwrags)>,
 <function __main__.Register.__call__.<locals>.wrapper(*args, **kwrags)>)

(<function __main__.Register.__call__.<locals>.wrapper(*args, **kwrags)>, 5)

fname	callcount
f:	2
q:	3



### Обработка исключений

**Исключения** (exceptions) - ещё один тип данных в python. Исключения необходимы для того, чтобы сообщать программисту об ошибках.

In [20]:
1 / 0

ZeroDivisionError: division by zero

`ZeroDivisionError` - название исключения

`division by zero` - краткое описание исключения

Избавиться от такой ошибки можно, написав `if`

In [26]:
d = 0
k = 1
if d:
    print(k / d)
else:
    k = 0
    print(k)

0


Но также можно написать обработку исключения:

In [25]:
try:
    k = 1 / 0
except ZeroDivisionError:
    k = 0

In [27]:
try:
    var = input()
    int_var = int(var)
except ValueError:      # ловит исключение типа ValueError 
    print('Это не число. Выходим.')
except Exception:       # ловит исключение любого типа 
    print('Это что ещё такое?')
else:                   # вызывается в случае, если исключение не было брошено
    print('Всё хорошо.')
finally:                # вызывается в любом случае
    print('Программа завершена.')

ыыавваф
Это не число. Выходим.
Программа завершена.


In [30]:
try:
    1 / 0
except Exception as e:
    print(type(e).__name__)
    print(e)

ZeroDivisionError
division by zero


In [32]:
try:
    raise Exception("Some exception")
except Exception as e:
    print("Exception exception " + str(e))

Exception exception Some exception


**Задача** Написать функцию вычисления среднего арифметического элементов переданного ей списка. Реализовать обработку возможных исключений при ее работе.